<a href="https://colab.research.google.com/github/stevejj4/SHOFCO-SUN/blob/master/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# Notebook 1: Load and Prepare Data

import pandas as pd

# Step 1: Define file paths
members_path = '/content/drive/MyDrive/Report processing test/Members.csv'
payments_path = '/content/drive/MyDrive/Report processing test/PaymentsJan.xlsx'

# Step 2: Load datasets
members = pd.read_csv(members_path)
payments = pd.read_excel(payments_path)  # Use pd.read_excel for Excel files

# Step 3: Convert necessary columns to numeric
# Ensure 'ID Number', 'NationalID' in members and 'Account no' in payments are numeric
members['ID Number'] = pd.to_numeric(members['ID Number'], errors='coerce')
members['NationalID'] = pd.to_numeric(members['ID Number'], errors='coerce')  # Assuming 'NationalID' is derived from 'ID Number'
payments['Account no'] = pd.to_numeric(payments['Account no'], errors='coerce')

print("Members and Payments data loaded and prepared successfully.")


<ipython-input-58-4ad21fbc92e3>:10: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  members = pd.read_csv(members_path)


Members and Payments data loaded and prepared successfully.


In [59]:
# Notebook 2: Merge Datasets

# Merge datasets on 'NationalID' and 'Account no'
merged_data = pd.merge(members, payments, left_on='NationalID', right_on='Account no')

print("Datasets merged successfully.")
# merged_data.head().


Datasets merged successfully.


In [60]:
# Notebook 3: Group and Aggregate Data

# Group by 'Group Name' and 'Ward', and calculate aggregations
grouped_data = merged_data.groupby(['Group Name', 'Ward']).agg(
    Total_Amount_Paid=('AMOUNT PAID', 'sum'),
    Total_Members_Paid=('NationalID', 'nunique')  # Count unique NationalIDs
).reset_index()

# Calculate total members in each group
member_counts = members.groupby('Group Name')['NationalID'].nunique().reset_index()
member_counts.columns = ['Group Name', 'Total Member Count']

# Merge aggregated data with member counts
final_group_totals = pd.merge(grouped_data, member_counts, on='Group Name', how='left')

# Reorder columns for readability
final_group_totals = final_group_totals[['Group Name', 'Ward', 'Total Member Count', 'Total_Members_Paid', 'Total_Amount_Paid']]

print("Group and aggregate data processed successfully.")
final_group_totals.head()


Group and aggregate data processed successfully.


,Group Name,Ward,Total Member Count,Total_Members_Paid,Total_Amount_Paid
0,10 Smart,Kabondo West,3,2,200
1,109 SHG,Lindi,12,12,3600
2,11C,Kware,2,2,300
3,12 Sisters,Utalii,26,10,1200
4,12 Sisters,Woodley,20,2,200


In [54]:
# Notebook 4: Export Results

# Export to Excel
output_path = '/content/drive/MyDrive/Report processing test/group_name_ward_member_size_paid.xlsx'
final_group_totals.to_excel(output_path, index=False)

print(f"Results exported successfully to {output_path}")


Results exported successfully to /content/drive/MyDrive/Report processing test/group_name_ward_member_size_paid.xlsx


In [63]:
# Install the necessary packages
!pip install chatterbot
!pip install chatterbot_corpus

#Import the chatbot libraries
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

# Create a new chatbot instance
chatbot = ChatBot('Data Insights Chatbot')

# Define some sample conversations
trainer = ListTrainer(chatbot)
trainer.train([
    "What is the total amount paid by Group A?",
    "The total amount paid by Group A is $10,000.",
    "How many members have paid in Ward X?",
    "50 members have paid in Ward X.",
])

# Get a response from the chatbot
response = chatbot.get_response("What is the average payment?")
print(response)

  Using cached ChatterBot-1.0.5-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached mathparse-0.1.2-py3-none-any.whl.metadata (776 bytes)
  Using cached Pint-0.24.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached pymongo-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached python_dateutil-2.7.5-py2.py3-none-any.whl.metadata (7.5 kB)
  Using cached PyYAML-5.1.2.tar.gz (265 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached chatterbot_corpus-1.2.0-py2.py3-none-any.whl.metadata (5.0 kB)
  Using 

ModuleNotFoundError: No module named 'chatterbot'